In [1]:
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt

%matplotlib inline
if 'google.colab' in str(get_ipython()):
  !pip3 install note_seq pretty_midi
  %run drive/My\ Drive/agent-based-tonal-model/tonal-agent-model.ipynb
else:
  %run tonal-agent-model.ipynb
  %run utility-functions.ipynb

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
####################################
# UTILITY FUNCTIONS
####################################

def write_cons(df, outpath):
    cons_df = pd.DataFrame()
    for w in [5,10,20]:
        cons_w = lagged_consonance(df,0,w)
        cons_df = pd.concat((cons_df,cons_w))
    cons_df.to_csv(outpath,index=False)
    return

'''
WINDOW_SIZE = 5
def gapped_consonance(notes, gap):
    extended_window_size = gap+WINDOW_SIZE*2
    cons_gap = [None]*len(notes)
    for i in range(extended_window_size,len(notes)):
        set1 = notes.iloc[(i-extended_window_size):(i-WINDOW_SIZE-gap)]
        set2 = notes.iloc[(i-WINDOW_SIZE):i]
        notes_comb = pd.concat((set1.notesA,set1.notesB,set2.notesA,set2.notesB)).values
        cc = individual_consonance_set(notes_comb)
        cons_gap[i] = cc
    return cons_gap
'''

def gapped_consonance(notes, gap, window):
    extended_window_size = gap+window*2
    cons_gap = [None]*len(notes)
    for i in range(extended_window_size,len(notes)):
        set1 = notes.iloc[(i-extended_window_size):(i-window-gap)]
        set2 = notes.iloc[(i-window):i]
        pc_hist1 = note_list_to_pc_hist(pd.concat((set1.notesA,set1.notesB)).values)
        pc_hist2 = note_list_to_pc_hist(pd.concat((set2.notesA,set2.notesB)).values)
        cons_gap[i] = get_cons_btw(pc_hist1, pc_hist2)
    return cons_gap

def write_gapped_cons(notes, gaps, windows, outpath):
    cons_gap_df = pd.DataFrame()
    for gap in gaps:
        for window in windows:
            cc = gapped_consonance(notes,gap,window)
            df = pd.DataFrame({'t': range(len(cc)),'cc': cc, 'gap': gap, 'window': window})
            cons_gap_df = pd.concat((cons_gap_df, df))
    cons_gap_df.to_csv(outpath, index=False)
    return

Simulation conditions.
* temperature [random, deterministic]
* memory [1 to very high]
* seeded vs unseeded
* coupled vs oneway
* windowed and temporal discounting memory implementations

1. Unseeded (notes + cons). Window, then temp decay
2. gapped cons
3. Seeded (notes + cons). Window, then temp decay
4. gapped cons 

temps = [0,1,10,20,100] # "deterministic" is also implemented
temps = [5,10,15]
windowed_mems = [1,5,20,50,100]
decay_mems = [1,5,10,50]
decay_mems = [1,2,3,4,5]
NUM_TRIALS = 5

# Simulate agents maximizing consonance with variable memory
(no self listening)

In [5]:
#OUTDIR = "output-max-cons-window/"

def simulate_coupled(m, b, seeded, outname):
    A = TonalAgent(memory_=m, gamma_=b)
    B = TonalAgent(memory_=m, gamma_=b)
    if seeded:
        A.seed()
        B.seed()
    for i in range(500):
        noteA = A.generate_next_note_max_cons_no_self()
        noteB = B.generate_next_note_max_cons_no_self()
        A.listen(noteB)
        B.listen(noteA)
    write_midi_file(A.prev_notes,OUTDIR+"midi/"+outname+'-playerA.mid')
    write_midi_file(B.prev_notes,OUTDIR+"midi/"+outname+'-playerB.mid')
    return pd.DataFrame({'t':range(len(A.prev_notes)),
                         'notesA': A.prev_notes,
                         'notesB': B.prev_notes})

# overdubbed duo
def run_overdubbed(ghost_notes,m,b,seeded,outname):
    # ghost_path: ghost_notes = pd.read_csv(ghost_path)
    ghost_notes = ghost_notes['notesB'].values
    A = TonalAgent(memory_=m, gamma_=b)
    if seeded: A.seed()
    for i in range(500):
        noteA = A.generate_next_note_max_cons_no_self()
        A.listen(ghost_notes[i])
    write_midi_file(A.prev_notes,OUTDIR+"midi/"+outname+'-live.mid')
    write_midi_file(ghost_notes,OUTDIR+"midi/"+outname+'-ghost.mid')
    return pd.DataFrame({'t':range(len(A.prev_notes)),
                         'notesA': A.prev_notes,
                         'notesB': ghost_notes})

In [3]:
!mkdir output-1step-mem
!mkdir output-1step-mem/notes
!mkdir output-1step-mem/consonance
!mkdir output-1step-mem/lagged-cons

In [4]:
OUTDIR = "output-1step-mem/"
####################################
# Unseeded Simulations
####################################
for b in [5,10,20]:
    for m in [1]:
        for i in range(20):
            print(i)
            # unseeded coupled
            outname = "coupled-unseeded-m"+str(m)+"-b"+str(b)+'-trial'+str(i)
            df = simulate_coupled(m,b,False,outname)
            df.to_csv(OUTDIR+"notes/"+outname+".csv",index=False)
            #write_gapped_cons(df, gaps, OUTDIR+"gapped-cons/"+outname+'.csv')
            write_cons(df,OUTDIR+"consonance/"+outname+".csv")
            
            # unseeded oneway
            df = run_overdubbed(df,m,b,False,outname.replace('coupled','oneway'))
            df.to_csv(OUTDIR+"notes/"+outname.replace('coupled','oneway')+'.csv',index=False)
            write_cons(df,OUTDIR+"consonance/"+outname.replace('coupled','oneway')+".csv")
            #write_gapped_cons(df, gaps, OUTDIR+"gapped-cons/"+outname.replace('coupled','oneway')+'.csv')

0


NameError: name 'simulate_coupled' is not defined

####################################
# Seeded Simulations
####################################
for b in temps:
    for m in windowed_mems:
        for i in range(NUM_TRIALS):
            # seeded coupled
            print('seeded coupled')
            outname = "coupled-seeded-m"+str(m)+"-b"+str(b)+'-trial'+str(i)
            df = simulate_coupled(m,b,True,outname)
            df.to_csv(OUTDIR+"notes/"+outname+".csv",index=False)
            write_cons(df,OUTDIR+"consonance/"+outname+".csv")
            
            # seeded oneway
            print('seeded oneway')
            df = run_overdubbed(df,m,b,True,outname.replace('coupled','oneway'))
            df.to_csv(OUTDIR+"notes/"+outname.replace('coupled','oneway')+'.csv',index=False)
            write_cons(df,OUTDIR+"consonance/"+outname.replace('coupled','oneway')+".csv")

## Simulate agents with temporal decay

Goal here is to manually tune the tau and softmax parameters. I really want to get those tonal basins!

!mkdir output-max-cons-time-decay-sanity-check
!mkdir output-max-cons-time-decay-sanity-check/notes
!mkdir output-max-cons-time-decay-sanity-check/midi
!mkdir output-max-cons-time-decay-sanity-check/gapped-cons
!mkdir output-max-cons-time-decay-sanity-check/consonance

In [3]:
OUTDIR = "output-max-cons-time-decay/"
if 'google.colab' in str(get_ipython()):
  OUTDIR = "drive/My Drive/agent-based-tonal-model/"+OUTDIR

def simulate_coupled(tau, b, seeded, outname):
    A = TonalAgent(tau_=tau, gamma_=b)
    B = TonalAgent(tau_=tau, gamma_=b)
    if seeded:
        A.seed()
        B.seed()
    for i in range(500):
        noteA = A.generate_next_note_temporal_decay()
        noteB = B.generate_next_note_temporal_decay()
        A.listen(noteB)
        B.listen(noteA)
    write_midi_file(A.prev_notes,OUTDIR+"midi/"+outname+'-playerA.mid')
    write_midi_file(B.prev_notes,OUTDIR+"midi/"+outname+'-playerB.mid')
    return pd.DataFrame({'t':range(len(A.prev_notes)),
                         'notesA': A.prev_notes,
                         'notesB': B.prev_notes})

# overdubbed duo
def run_overdubbed(ghost_notes,tau,b,seeded,outname):
    # ghost_path: ghost_notes = pd.read_csv(ghost_path)
    ghost_notes = ghost_notes['notesB'].values
    A = TonalAgent(tau_=tau, gamma_=b)
    if seeded: A.seed()
    for i in range(500):
        noteA = A.generate_next_note_temporal_decay()
        A.listen(ghost_notes[i])
    write_midi_file(A.prev_notes,OUTDIR+"midi/"+outname+'-live.mid')
    write_midi_file(ghost_notes,OUTDIR+"midi/"+outname+'-ghost.mid')
    return pd.DataFrame({'t':range(len(A.prev_notes)),
                         'notesA': A.prev_notes,
                         'notesB': ghost_notes})

In [10]:
gaps = [1,5,10,25,50,100]
windows = [5]
def run_simulation(tau, gamma, beg_trial, end_trial):
    for i in range(beg_trial,end_trial):
        print(i)
        # unseeded coupled
        outname = "coupled-unseeded-tau"+str(tau)+"-b"+str(gamma)+'-trial'+str(i)
        df = simulate_coupled(tau,gamma,False,outname)
        df.to_csv(OUTDIR+"notes/"+outname+".csv",index=False)
        #write_gapped_cons(df, gaps, windows, OUTDIR+"gapped-cons/"+outname+'.csv')
        write_cons(df,OUTDIR+"consonance/"+outname+".csv")
            
        # unseeded oneway
        df = run_overdubbed(df,tau,gamma,False,outname.replace('coupled','oneway'))
        df.to_csv(OUTDIR+"notes/"+outname.replace('coupled','oneway')+'.csv',index=False)
        write_cons(df,OUTDIR+"consonance/"+outname.replace('coupled','oneway')+".csv")
        #write_gapped_cons(df, gaps, windows, OUTDIR+"gapped-cons/"+outname.replace('coupled','oneway')+'.csv')
    return
        
run_simulation(.1,1,0,20)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [8]:
np.unique([f.split('/')[-1].split('-')[0:3].join() for f in glob("output-max-cons-time-decay/notes/*.csv")])

AttributeError: 'list' object has no attribute 'join'

####################################
# Unseeded Simulations
####################################
for b in temps:
    for tau in decay_mems:
        for i in range(5,20):
            print(i)
            # unseeded coupled
            outname = "coupled-unseeded-tau"+str(tau)+"-b"+str(b)+'-trial'+str(i)
            df = simulate_coupled(tau,b,False,outname)
            df.to_csv(OUTDIR+"notes/"+outname+".csv",index=False)
            #write_gapped_cons(df, gaps, windows, OUTDIR+"gapped-cons/"+outname+'.csv')
            write_cons(df,OUTDIR+"consonance/"+outname+".csv")
            
            # unseeded oneway
            df = run_overdubbed(df,tau,b,False,outname.replace('coupled','oneway'))
            df.to_csv(OUTDIR+"notes/"+outname.replace('coupled','oneway')+'.csv',index=False)
            write_cons(df,OUTDIR+"consonance/"+outname.replace('coupled','oneway')+".csv")
            #write_gapped_cons(df, gaps, windows, OUTDIR+"gapped-cons/"+outname.replace('coupled','oneway')+'.csv')

## Get Gapped Consonanace Measures

from glob import glob
OUTDIR = "output-max-cons-time-decay/"
notes_files = [f for f in glob(OUTDIR+"notes/*.csv") if 'tau50-b100' in f]
len(notes_files)

gaps = [1,5,10,50,100]
windows = [5,10]
temps = [5,10,15]
taus = [1,2,3,4,5]
notes_files = [f for f in glob(OUTDIR+"notes/*.csv") if 'tau50-b100' in f]
for f in notes_files:
    #tau = int(f.split("-")[-3].replace('tau',''))
    #temp = int(f.split("-")[-2].replace('b',''))
    #if (not temp in temps) or (not tau in taus): continue
    print(f)
    notes = pd.read_csv(f)
    write_gapped_cons(notes, gaps, windows, OUTDIR+"gapped-cons/"+f.split('/')[-1])        

## Get lagged-consonance files for all notes files

from glob import glob

files = glob('output-max-cons/notes/*.csv')
lagged_files = glob('output-max-cons/lagged-consonance/*.csv')
for f in files:
    #df = pd.read_csv(f)
    outpath = OUTDIR+'lagged-consonance/'+f.split('/')[-1]
    if outpath in lagged_files: continue
    print(f)
    lagged_df = pd.DataFrame()
    df = pd.read_csv(f)
    for lag in [-10,-5,0,5,10]:
        print(lag)
        lagged_df = pd.concat((lagged_df,lagged_consonance(df,lag,5)))
    lagged_df.to_csv(outpath,index=False)

#############################
# Utility functions for running
# simulations
#############################

SEED_B = 10 # for seeded simulations
NUM_ITERATIONS = 500

# mutually coupled duo
def run_coupled_duo(m, b, seeded, outname):
    A = TonalAgent(memory_=m,base_=b)
    B = TonalAgent(memory_=m,base_=b)
    if seeded:
        A.seed()
        B.seed()
    for i in range(NUM_ITERATIONS):
        noteA = A.generate_next_note_listen_no_self() # change this for listen to self
        noteB = B.generate_next_note_listen_no_self()
        A.listen(noteB)
        B.listen(noteA)
    write_midi_file(A.prev_notes,outname+'-playerA.mid')
    write_midi_file(B.prev_notes,outname+'-playerB.mid')
    return pd.DataFrame({'t':range(len(A.prev_notes)),
                         'notesA': A.prev_notes,
                         'notesB': B.prev_notes})

# solo
def run_solo(m,b,seeded,outname):
    A = TonalAgent(memory_=m,base_=b)
    if seeded: A.seed()
    for i in range(1000):
        A.generate_next_note()
    write_midi_file(A.prev_notes,outname+'.mid')
    cons_df = individual_consonance(A.prev_notes,outpath=outname+'-consonance.csv')
    return cons_df

# overdubbed duo
def run_overdubbed(ghost_path,m,b,seeded,outname):
    ghost_notes = pd.read_csv(ghost_path)
    ghost_notes = ghost_notes['notesB'].values
    A = TonalAgent(memory_=m,base_=b)
    if seeded: A.seed()
    for i in range(NUM_ITERATIONS):
        noteA = A.generate_next_note_listen_no_self()
        A.listen(ghost_notes[i])a
    write_midi_file(A.prev_notes,outname+'-live.mid')
    write_midi_file(ghost_notes,outname+'-ghost.mid')
    return pd.DataFrame({'t':range(len(A.prev_notes)),
                         'notes-live': A.prev_notes,
                         'notes-ghost': ghost_notes})

## Simulate coupled duos

%run tonal-agent-model.ipynb

m = 1
for b in [1,100]:
    for i in range(20):
        print(i)
        outname = "output/notes/coupled-seeded-m"+str(m)+"-b"+str(b)+'-trial'+str(i)
        df = run_coupled_duo(m,b,True,outname)
        df.to_csv(outname+".csv",index=False)
        outname = outname.replace("seeded","unseeded")
        df = run_coupled_duo(m,b,False,outname)
        df.to_csv(outname+".csv",index=False)

## Simulate overdubbed duos

m = 1
for b in [1,100]:
    for i in range(20):
        print(i)
        outname = "output/notes/oneway-seeded-m"+str(m)+"-b"+str(b)+'-trial'+str(i)
        ghost_path = outname.replace("oneway","coupled")+'.csv'
        df = run_overdubbed(ghost_path, m, b, True, outname)
        df.to_csv(outname+".csv",index=False)
        outname = outname.replace("seeded","unseeded")
        ghost_path = outname.replace("oneway","coupled")+'.csv'
        df = run_overdubbed(ghost_path, m, b, False, outname)
        df.to_csv(outname+".csv",index=False)

## Analyze coupled vs overdubbed simulations

simulation_files = glob("output-max-cons/consonance/*.csv")
for f in simulation_files:
    if not 'oneway' in f: continue
    print(f)
    notes_df = pd.read_csv(f)
    df = combined_consonance(notes_df.iloc[:,1],
                        notes_df.iloc[:,2])
    outname = f.split('/')[-1]
    df.to_csv("output-max-cons/consonance/"+outname, index=False)

In [ ]:
simulation_files[0]

notes_df.iloc[:500,1:]

## Debug -- does seeding work?

# debug -- does seeding work?

seed_cons = []
rand_cons = []
for i in range(500):
    seed_cons.append(individual_consonance_set(TonalAgent().seed()))
    rand_cons.append(individual_consonance_set(np.random.choice(12,50)))

plt.hist(seed_cons,alpha=.2)
plt.hist(rand_cons,alpha=.2)

In [ ]:
print(np.mean(seed_cons))
print(np.mean(rand_cons))

Yes, it appears to work. Average random consonance is -1.55 -- it appears to saturate at very low values around -1.91.

In [ ]:
len(coupled_df)/3

## Seeded solo agents

In [ ]:
%run tonal-agent-model.ipynb

In [ ]:
A = TonalAgent(memory_=1,base_=10)

In [ ]:
def run_solo(m,seeded=False,outname='blah'):
    playerA = TonalAgent(memory_=m,base_=10)
    if seeded: A.seed()
    for i in range(1000):
        A.generate_next_note()
    write_midi_file(A.prev_notes,outname+'.mid')
    cons_df = individual_consonance(A.prev_notes,outpath=outname+'-consonance.csv')
    return cons_df

# cons_df = pd.DataFrame()
for m in [1,5,20,100]:
    print(m)
    for i in range(20):
        df = run_solo(m,seeded=True)
        df['t'] = range(len(df))
        df['memory'] = m
        df['trial'] = i
        cons_df = pd.concat((cons_df,df))

In [ ]:
cons_df['softmax_b'] = 10
cons_df.to_csv('output/seeded-solo.csv',index=False)

In [ ]:
cons_df.groupby(['memory']).mean()

No apparent difference in means.

In [ ]:
plt.plot(cons_df.cA,'.')
plt.show()
plt.plot(cons_df.rolling(20).cA.mean(),'.')
plt.show()

In [ ]:
for m in np.unique(cons_df['memory']):
    print(m)
    cons_df_m = cons_df[cons_df.memory==m]
    for i in np.unique(cons_df_m['trial']):
        cons_trial = cons_df_m[cons_df_m.trial==i]
        plt.plot(cons_trial['t'],cons_trial['cA'],'.',alpha=.2)
    plt.plot(cons_df_m.rolling(50).cA.mean(),'.')
    plt.show()

looks like things are pretty much the same for these different memories. i expected that consonance would drop off for lag-1. hmm. what does make sense is that consonance starts high, decreases after about 100 gens, then evens out for the rest of the time. maybe I should consider random too, as a baseline?

## Compute Lagged Consonance

In [ ]:
from glob import glob

window = 5
files = glob('output-max-cons/consonance/*.csv')
for f in files:
    print(f)
    df = pd.read_csv(f)
    cons = pd.DataFrame()
    for lag in [-10,-5,0,5,10]:
        lagged_cons = lagged_consonance(df,lag,window)
        cons = pd.concat((cons,lagged_cons))
    cons.to_csv('output-max-cons/lagged-consonance/'+f.split('/')[-1],index=False)